In [2]:
!pip install tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 1.0 MB/s eta 0:00:00 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 1.3 MB/s eta 0:00:00m eta 0:00:01m0:00:21
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 3.4 MB/s eta 0:00:00m eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 3.1 MB/s eta 0:00:00m eta 0:00:010:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 4.1 MB/s eta 0:00:00m eta 0:00:010:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 5.3 MB/s eta 0:00:005.7 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 7.0 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 6.5 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 7.4 MB/s eta 0:0

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from sklearn.metrics import accuracy_score

# Load a subset of MNIST dataset
(x_train, y_train), _ = mnist.load_data()
x_subset = x_train[:1000].reshape(-1, 28*28) / 255.0  # Use the first 1000 samples

# Implement Incremental K-means algorithm from scratch
def incremental_kmeans(X, k, batch_size=100, max_iters=100):
    # Initialize centroids using random data points
    centroids = X[np.random.choice(len(X), k, replace=False)]
    
    for _ in range(max_iters):
        # Shuffle the data
        np.random.shuffle(X)
        
        # Divide data into batches
        for i in range(0, len(X), batch_size):
            batch = X[i:i+batch_size]
            
            # Assign each data point in the batch to the nearest centroid
            distances = np.linalg.norm(batch[:, np.newaxis] - centroids, axis=2)
            labels = np.argmin(distances, axis=1)
            
            # Update centroids based on the batch
            for j in range(k):
                centroid_points = batch[labels == j]
                if len(centroid_points) > 0:
                    centroids[j] = np.mean(centroid_points, axis=0)

    return centroids

# Applying Incremental K-means
k = 10  # Number of clusters (since there are 10 digits)
centroids = incremental_kmeans(x_subset, k)

# Assign labels to centroids based on majority class
centroid_labels = []
for centroid in centroids:
    distances = np.linalg.norm(x_subset - centroid, axis=1)
    closest_labels = y_train[np.argsort(distances)[:10]]  # Select top 10 closest data points
    centroid_label = np.argmax(np.bincount(closest_labels))
    centroid_labels.append(centroid_label)

print("Centroid Labels:", centroid_labels)

# Calculate accuracy score
y_pred = [centroid_labels[np.argmin(np.linalg.norm(centroid - centroids, axis=1))] for centroid in x_subset]
accuracy = accuracy_score(y_train[:1000], y_pred)
print("Accuracy Score:", accuracy)


2024-04-05 09:55:14.169755: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-05 09:55:14.239159: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-05 09:55:14.490810: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-05 09:55:16.532319: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Centroid Labels: [9, 0, 5, 1, 1, 7, 9, 0, 2, 7]
Accuracy Score: 0.098
